In [7]:
# default_exp data.format_qtmdata

In [8]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Format qtmdata
>  Format the Qualisys sports data preprocessed from Matlab 

In [9]:
#export
import numpy as np
import pandas as pd
import argparse
import os

In [10]:
#export
def parse_args():
    parser = argparse.ArgumentParser(
        description='Reformat qtmdata so that we can train.'
    )
    parser.add_argument(
        '--data-file', 
        dest='data_file',
        help='qtm text file that has been formated in matlab', 
        type=str
    )
    parser.add_argument(
        '--output-dir',
        dest='output_dir',
        help='directory for reformated keypoint data (default: ./)',
        default='./',
        type=str
    )
    
    return parser.parse_args()

In [11]:
#export
def main(args):
    """
    Loads the qtm data then removes unwanted keypoints.
    Then it infers new keypoints adds them. 
    Further we scale the dataset using a the norm vector between the 
    root and 'SpineThoracic2'. 
    """
    # Loads the textfiles
    labels_np = np.loadtxt('qtm_labels_py.txt', dtype = 'str')
    data_3D = np.loadtxt(args.data_file, dtype = 'float', delimiter= ',')

    # Reformats the data to a dataframe
    data_3D = pd.DataFrame(data_3D, index=labels_np).T

    # Remove unwanted keypoints
    data_3D = data_3D.drop(
        columns=[
            'HeadL', 'HeadR', 'Chest', 'LThighFrontLow', 'RThighFrontLow', 
            'LShinFrontHigh', 'RShinFrontHigh', 'LForefoot5', 'RForefoot5', 
            'LHeelBack', 'RHeelBack', 'LArm', 'RArm','WaistLFront', 'WaistL', 
            'WaistRFront', 'WaistR'
        ]
    )
    # Create "new" keypoints by finding the mean between some specific keypoints
    left_elbow_3D = data_3D.loc[:, ['LElbowOut','LElbowIn']].mean(axis=1)
    right_elbow_3D = data_3D.loc[:, ['RElbowOut','RElbowIn']].mean(axis=1)

    left_wrist_3D = data_3D.loc[:, ['LWristIn','LWristOut']].mean(axis=1)
    right_wrist_3D = data_3D.loc[:, ['RWristOut','RWristIn']].mean(axis=1)

    left_knee_3D = data_3D.loc[:, ['LKneeOut','LKneeIn']].mean(axis=1)
    right_knee_3D = data_3D.loc[:, ['RKneeOut','RKneeIn']].mean(axis=1)

    #left_waist_3D = data_3D.loc[:, ['WaistLFront','WaistL']].mean(axis=1)
    #right_waist_3D = data_3D.loc[:, ['WaistRFront','WaistR']].mean(axis=1)

    left_ankle_3D = data_3D.loc[:, ['LAnkleOut','LAnkleIn']].mean(axis=1)
    right_ankle_3D = data_3D.loc[:, ['RAnkleOut','RAnkleIn']].mean(axis=1)

    # Remove the keypoints that was taken as a mean
    data_3D = data_3D.drop(
        columns=[
            'LElbowOut','LElbowIn', 'RElbowOut','RElbowIn', 
            'LWristIn','LWristOut', 'RWristIn','RWristOut', 
            'LKneeIn', 'LKneeOut','RKneeIn', 'RKneeOut',
            'LAnkleOut','LAnkleIn','RAnkleOut','RAnkleIn'
        ]
    )
    # Adds the new keypoint data to the dataframe
    data_3D['LElbow'] = left_elbow_3D
    data_3D['RElbow'] = right_elbow_3D
    data_3D['LWrist'] = left_wrist_3D
    data_3D['RWrist'] = right_wrist_3D
    data_3D['LKnee'] = left_knee_3D
    data_3D['RKnee'] = right_knee_3D
    #data_3D['LWaist'] = left_waist_3D
    #data_3D['RWaist'] = right_waist_3D
    data_3D['LAnkle'] = left_ankle_3D
    data_3D['RAnkle'] = right_ankle_3D

    # Convert all the data relative to the root 'WaistBack'
    #data_3D = data_3D.subtract(data_3D['WaistBack'], axis=0)
    
    # Scale each frame with a norm vector
    #data_3D_scaled = [] 
    #for i in range(0, data_3D.shape[0], 3):
        # Calculates the scale factor for each frame
        #norm_vector = np.sqrt(np.square(
            #data_3D['SpineThoracic2'].iloc[i:i+3]).sum(axis=0)
        #)
        #data_3D_scaled.append(data_3D.iloc[i:i+3].divide(norm_vector))

    # Reformat to dataframe again after all data has been scaled
    #data_3D = pd.concat(data_3D_scaled, ignore_index=True)

    # Remove the y-dimension to get the 2D data for side cam. 
    # OBS! This may vary between camera angels.
    # TODO: Add argument argument for which dim to drop to 2D
    data_2D = data_3D.drop(index=range(1, data_3D.shape[0], 3))
    
    # Creates output names that depends on the name of the data file 
    data_file_name = os.path.basename(os.path.normpath(args.data_file)).rsplit(".")[0]
    out_2D = os.path.join(args.output_dir, data_file_name + '_2D_keypoints.csv')
    out_3D = os.path.join(args.output_dir, data_file_name + '_3D_keypoints.csv')
    
    # Save the keypoint data as csv files
    pd.DataFrame.to_csv(data_2D, path_or_buf=out_2D)
    pd.DataFrame.to_csv(data_3D, path_or_buf=out_3D)

In [12]:
#export
try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False

if __name__ == '__main__' and not IN_NOTEBOOK:
    args = parse_args()
    main(args)

usage: ipykernel_launcher.py [-h] [--data-file DATA_FILE]
                             [--output-dir OUTPUT_DIR]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9011 --control=9009 --hb=9008 --Session.signature_scheme="hmac-sha256" --Session.key=b"f8abbca2-3eec-47de-950e-f847a25a676a" --shell=9010 --transport="tcp" --iopub=9012 --f=C:\Users\Familjen\AppData\Local\Temp\tmp-9164POGX1yT326mU.json


SystemExit: 2

C:\Users\Familjen\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_model.ipynb.
Converted 01_loss.ipynb.
Converted 02_skeleton.ipynb.
Converted 03_mocap_dataset.ipynb.
Converted 04_h36m_dataset.ipynb.
Converted 05_camera.ipynb.
Converted 06_quaternion.ipynb.
Converted 07_utils.ipynb.
Converted 08_generators.ipynb.
Converted 09_custom_dataset.ipynb.
Converted 10_visualization.ipynb.
Converted 11_arguments.ipynb.
Converted 12_data_utils.ipynb.
Converted 13_prepare_data_2d_custom.ipynb.
Converted 14_infer_video.ipynb.
Converted 15_prepare_data_COCO.ipynb.
Converted 16_pycococreatortools.ipynb.
Converted 17_format_qtmdata.ipynb.
No export destination, ignored:
#export
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Import some common libraries
import numpy as np
import matplotlib.pyplot as plt
import os, json, cv2, random

# Import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.conf

AttributeError: 'NoneType' object has no attribute 'start'